In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
import sys
import codecs
import re
from torch.utils.data import DataLoader

def trans(test_docs, towords=True):
    doc1 = []
    doc2 = []
    for line in codecs.open(test_docs, 'r', 'utf-8').readlines():
        doc1.append(line[0])
        doc2.append(line[1])
    if towords:
        doc1 = [sen.split() for sen in doc1]
        doc2 = [sen.split() for sen in doc2]    
    return doc1, doc2

def train_trans(lines):
    doc1 = []
    doc2 = []
    s1=str(lines).split('\\n')
    for i in range(0,len(s1)-1):
      doc1.append(s1[i].split('\\t')[0])
      doc2.append(s1[i].split('\\t')[1])
    return doc1, doc2

def train_d_trans(train_docs):
    for sen in range(len(train_docs)):
        train_docs[sen] = train_docs[sen].strip('.').strip('?').strip(':').strip(',').strip(')').strip('(')
    return train_docs

def txt_to_list(gs):
    lst=gs.split('\n')
    print(lst)
    return lst
    
def merge(file1, file2):
    f1 = open(file1, 'a+', encoding='utf-8')
    with open(file2, 'r', encoding='utf-8') as f2:
        for i in f2:
            f1.write(i)
    f1.close()

from math import sqrt

def dot_product(v1, v2):
    return sum(a * b for a, b in zip(v1, v2))

def magnitude(vector):
    return sqrt(dot_product(vector, vector))

def similarity(v1, v2):
    return dot_product(v1, v2) / (magnitude(v1) * magnitude(v2) + .00000000001)
from sentence_transformers import SentenceTransformer,InputExample

import torch
from torch import nn, Tensor
from typing import Iterable, Dict
from sentence_transformers import SentenceTransformer


class CosineSimilarityLoss(nn.Module):
    
    def __init__(self, model: SentenceTransformer, loss_fct = nn.MSELoss(), cos_score_transformation=nn.Identity()):
        super(CosineSimilarityLoss, self).__init__()
        self.model = model
        self.loss_fct = loss_fct
        self.cos_score_transformation = cos_score_transformation


    def forward(self, sentence_features: Iterable[Dict[str, Tensor]], labels: Tensor):
        embeddings = [self.model(sentence_feature)['sentence_embedding'] for sentence_feature in sentence_features]
        cosSimi = torch.cosine_similarity(embeddings[0], embeddings[1]) 
        output = self.cos_score_transformation(5*(cosSimi+1)/2) 
        return self.loss_fct(output, labels.view(-1))

model = SentenceTransformer('all-mpnet-base-v2')
model_save_path = 'model_all_mp_base_v2_e10_w1000_bs64/'

f= open("train_data.txt","r")
lines = str(f.readlines())
docs1, docs2 = train_trans(lines)
f= open("train_gs.txt","r")
gs = txt_to_list(f.read())
print(gs)

train_data = list(zip(docs1, docs2))
train_samples = []
# print(train_samples)
for i in range(len(train_data)):
    train_samples.append(InputExample(texts=train_data[i], label=float(gs[i])))

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=64)

train_loss = CosineSimilarityLoss(model)

model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=5,
          warmup_steps=100,
          output_path=model_save_path)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
['4.000', '3.750', '2.800', '3.400', '2.400', '1.333', '4.600', '3.800', '4.200', '2.600', '4.400', '4.200', '5.000', '5.000', '5.000', '3.400', '5.000', '4.000', '3.200', '5.000', '4.400', '3.600', '3.600', '0.800', '1.400', '3.200', '3.800', '4.200', '3.000', '3.600', '3.000', '3.000', '3.000', '4.800', '2.500', '2.250', '3.500', '3.500', '3.000', '3.000', '3.667', '1.667', '3.600', '3.800', '3.400', '3.400', '3.800', '2.800', '3.750', '4.400', '3.750', '2.750', '3.750', '2.250', '3.800', '3.200', '3.800', '4.800', '4.000', '1.400', '4.200', '1.800', '4.200', '2.000', '2.600', '3.250', '5.000', '3.000', '3.750', '4.250', '4.750', '3.250', '3.750', '3.250', '3.800', '5.000', '3.600', '3.800', '2.000', '3.400', '4.200', '3.400', '3.000', '3.000', '3.250', '2.750', '2.600', '3.400', '3.600', '3.600', '4.800', '4.000', '3.600', '3.400', '2.400', '4.222', '3.000', '3.000', '2.750', '4.000', 

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/35 [00:00<?, ?it/s]

Iteration:   0%|          | 0/35 [00:00<?, ?it/s]

Iteration:   0%|          | 0/35 [00:00<?, ?it/s]

Iteration:   0%|          | 0/35 [00:00<?, ?it/s]

Iteration:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
model1 = SentenceTransformer('model_all_mp_base_v2_e10_w1000_bs64')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
with open('STS2016.input.headlines.txt') as f:
    lines = str(f.readlines())
s1=str(lines).split('\\n')
line1=[]
line2=[]
for i in range(0,len(s1)-1):
  line1.append(s1[i].split('\\t')[0])
  line2.append(s1[i].split('\\t')[1])
import re
string1=[]
for i in line1:
  string1.append(re.sub(r"[^a-zA-Z0-9]+",' ',i))
string2=[]
for i in line2:
  string2.append(re.sub(r"[^a-zA-Z0-9]+",' ',i))
strin=string1 +string2


In [ ]:
import numpy as np
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

model1 = SentenceTransformer('model_all_mp_base_v2_e10_w1000_bs64')

x=0
similar=[]
for i in range(0, len(string1)-1):
  query = string1[i]
  query2 = string2[i]
  query_vec = model1.encode(query, convert_to_tensor=True)
  sim = cosine(query_vec, model1.encode(query2, convert_to_tensor=True))
  print("Sentence = ", string1[i], string2[i], "; similarity = ", sim)
  similar.append(sim)
  x=x+1


Sentence =   Digital era threatens tenuous future of drive ins Digital Era Threatens Future of Drive Ins ; similarity =  0.9717845
Sentence =   Jessica Lal murder convict Manu Sharma gets 15 day parole Jessica murder Manu Sharma gets parole ; similarity =  0.7751637
Sentence =   UN to hold emergency DR Congo talks U N Council to Hold Emergency DR Congo Talks ; similarity =  0.9623993
Sentence =   Iran and IAEA resume nuclear talks Iran IAEA resume nuclear talks in Tehran ; similarity =  0.93216133
Sentence =   Thai protesters storm army headquarters Thai protestors storm Royal army headquarters in Bangkok ; similarity =  0.88470894
Sentence =   40 Still Missing in Deadly Canada Oil Train  40 still missing in deadly Canada oil train crash ; similarity =  0.972842
Sentence =   Raghuram Rajan to step into eye of rupee storm Raghuram Rajan to step into eye of rupee storm as RBInbspguv tomorrow ; similarity =  0.9319148
Sentence =   Iran Foreign Ministry to lead nuclear talks Iran foreign m

In [ ]:
final=[]
for i in similar:
  final.append(i*5)
print(final)

[4.858922362327576, 3.8758185505867004, 4.811996519565582, 4.660806655883789, 4.423544704914093, 4.864209890365601, 4.659574031829834, 4.352940917015076, 4.891533255577087, 4.951597452163696, 4.898202419281006, 3.7056225538253784, 4.603016972541809, 4.798072576522827, 4.766883850097656, 5.0, 4.816564917564392, 4.71011608839035, 4.500108361244202, 4.41193163394928, 4.80012834072113, 4.526115953922272, 4.3204763531684875, 4.342384040355682, 4.871166348457336, 4.854991137981415, 4.820675253868103, 4.681219756603241, 4.88933652639389, 4.601179361343384, 4.831302464008331, 5.0, 4.565343260765076, 4.0762147307395935, 4.591798186302185, 4.321375787258148, 4.907357096672058, 4.767286777496338, 3.9368638396263123, 4.860296845436096, 3.8215258717536926, 4.897732734680176, 4.535013437271118, 4.45178896188736, 4.585085511207581, 3.850471079349518, 5.0, 4.923854172229767, 4.784665107727051, 3.650869131088257, 4.780849516391754, 4.666513502597809, 4.601414799690247, 4.984511733055115, 4.129885435104

In [ ]:
!zip -r /content/model_all_mp_base_v2_e10_w1000_bs64.zip /content/model_all_mp_base_v2_e10_w1000_bs64

  adding: content/model_all_mp_base_v2_e10_w1000_bs64/ (stored 0%)
  adding: content/model_all_mp_base_v2_e10_w1000_bs64/vocab.txt (deflated 53%)
  adding: content/model_all_mp_base_v2_e10_w1000_bs64/config_sentence_transformers.json (deflated 27%)
  adding: content/model_all_mp_base_v2_e10_w1000_bs64/eval/ (stored 0%)
  adding: content/model_all_mp_base_v2_e10_w1000_bs64/special_tokens_map.json (deflated 50%)
  adding: content/model_all_mp_base_v2_e10_w1000_bs64/pytorch_model.bin (deflated 8%)
  adding: content/model_all_mp_base_v2_e10_w1000_bs64/tokenizer_config.json (deflated 48%)
  adding: content/model_all_mp_base_v2_e10_w1000_bs64/README.md (deflated 53%)
  adding: content/model_all_mp_base_v2_e10_w1000_bs64/sentence_bert_config.json (deflated 4%)
  adding: content/model_all_mp_base_v2_e10_w1000_bs64/config.json (deflated 48%)
  adding: content/model_all_mp_base_v2_e10_w1000_bs64/1_Pooling/ (stored 0%)
  adding: content/model_all_mp_base_v2_e10_w1000_bs64/1_Pooling/config.json (d